IA & Data science (LU3IN026) -- 2020-2021
--------
*&copy; Equipe pédagogique: Vincent Guigue, Christophe Marsala, Edoardo Sarti, Olivier Schwander.*


## Projet 2021

### Préparation du notebook


<font size="+1" color="RED">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms :**

Ung Thierry

<font color="RED" size="+1">**[Q]**</font> **Renommer ce fichier ipython**

Tout en haut de cette page, cliquer sur <tt>projet-2021</tt> et rajouter à la suite de <tt>projet-2021</tt> les noms des membres du binômes séparés par un tiret.

Par exemple, pour le binôme Luke Skywalker et Han Solo, le nom de fichier devient `projet2021-Skywalker-Solo`

Penser à sauvegarder fréquemment le fichier en cours de travail :
- soit en cliquant sur l'icône "disquette"
- soit par la combinaison de touches [Ctrl]-S

## Données

Les données vous sont fournies sur le moodle. 
Ces données sont fournies sur Kaggle, ce sont les données *Google Play Store Apps* accessibles à l'adresse https://www.kaggle.com/lava18/google-play-store-apps.

Il est indispensable de lire en détail la page Kaggle pour comprendre à quoi ces données correspondent.

Le compte-rendu a fournir le jour de la dernière séance de TDTME de votre groupe doit comporter:
- un fichier PDF qui correspond à un poster sur lequel sont expliqués les différents problèmes traités, la façon dont ils ont été traités, et les résultats obtenus.
- un notebook par problème traité, vous pouvez traiter autant de problème que vous le souhaitez. Le problème étudié doit être décrit précisément et vous devez impérativement suivre le format ci-dessous.

Bien entendu, le tout sera mis dans un fichier archive (tar.gz ou zip exclusivement) et déposé sur le site Moodle.


## Imports

Dans cette partie du notebook, on effectue tout les import de modules allant etre utilisés dans le projet.

In [ ]:
# Importation des librairies standards :
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline  

# Importation de votre librairie iads :
# La ligne suivante permet de préciser le chemin d'accès à la librairie iads
import sys
sys.path.append('../') # iads doit être dans le répertoire frère du répertoire courant !

# Importation de la librairie iads
import iads as iads

# importation de Classifiers
from iads import Classifiers as cl

# importation de utils
from iads import utils as ut

# commande TRES utile pour recharger automatiquement le code que vous modifiez dans les modules
%load_ext autoreload
%autoreload 2

## Partie 0 - Chargement et nettoyage des données

## Données Google Play Store

Dans les données à analyser, il existe des exemples ne pouvant pas etre utilisés puisque certains champs de ces exemples sont inutilisables. 
Par exemple, il peut s'agir de champs Nan ou de valeurs anormales (dans le champ Size de certains exemples, on a parfois la valeur 'Varies with device'). 
Il faut donc retirer ces exemples de la base de données en priorité.

On récupère les données :

In [ ]:
gpsdf = pd.read_csv("data/GoogleApps/googleplaystore.csv")
gpsdf[0:5]

Le nombre d'exemples dans la base de données est de :

In [ ]:
print(gpsdf.shape[0], "exemples")

Les différents champs des exemples de cette base de données sont :

In [ ]:
for i in gpsdf.columns :
    print(i)
print("(" + str(len(gpsdf.columns)) + " champs différents)")

On affiche les informations relatives à chacun des champs de la base de données :

In [ ]:
gpsdf.info()

On remarque que les champs Rating, Type, Content Rating, Current Ver ou Android Ver de certains exemples sont nuls. Il faut donc retirer ces exemples en priorité.

On s'occupe dans un premier temps de retirer toutes les exemples possedant des champs avec des valeurs Nan.

In [ ]:
gpsdf = gpsdf.dropna()
gpsdf.info()

Le nombre d'exemples dans la base de données est maintenant de 9360 exemples.

La fonction suivante permet de retirer le prefixe, le suffixe et d'autres éléments inutiles dans les champs des exemples de la base de données.

In [ ]:
def clean_elements(column) :
    """ La fonction prend en argument une colonne de la base de données et la renvoie nettoyée
    """
    column = column.str.replace('+', '') # Suppression des symboles +
    column = column.str.replace(',', '') # Suppression des symboles ,
    column = column.str.replace('$', '') # Suppression des symboles $
    column = column.str.replace("'", '') # Suppression des symboles '
    
    return column

### Nettoyage de la colonne Category

On affiche les valeurs unique de la colonne Category.

In [ ]:
gpsdf.loc[:,'Category'].unique()

Afin que ces valeurs soient utilisables lors de l'apprentissage, on décide de diviser la colonne 'Category' en plusieurs colonnes différents. 
<br/>Chaque colonne permettra de savoir à quelle catégorie de 'Category' cette application appartient. 
<br/>(Par exemple, si une application a la valeur 1 dans son champ 'Category_GAME', cela signifie que l'application est destinée fait partie de la catégorie 'GAME').

In [ ]:
gpsdf = pd.concat([gpsdf, pd.get_dummies(gpsdf['Category'], prefix='C')], axis=1)
gpsdf.drop(['Category'], axis=1, inplace=True)

On affiche l'état de la base de données après avoir réalisé la modification.

In [ ]:
gpsdf.head()

### Nettoyage de la colonne Rating

On affiche les valeurs uniques de la colonne Rating. Il s'agit de flottants donc ces valeurs peut etre directement utilisées.

In [ ]:
gpsdf.loc[:,'Rating'].unique()

### Nettoyage de la colonne Reviews

On affiche les valeurs uniques de la colonne Reviews.

In [ ]:
gpsdf.loc[:,'Reviews'].unique()

On souhaite également transformer les valeurs de Reviews en entier afin de faciliter leurs utilisations lors de l'apprentissage (elles sont pour l'instant au format String).
<br/>On utilise pour cela la fonction to_numeric de la bibliothèque pandas.

In [ ]:
gpsdf['Reviews'] = pd.to_numeric(gpsdf['Reviews'])
gpsdf.loc[:,'Reviews'].unique()

### Nettoyage de la colonne Size

On affiche les valeurs uniques de la colonne Size.

In [ ]:
gpsdf.loc[:,'Size'].unique()

La valeur 'Varies with device' ne pouvant pas etre utilisée lors de l'apprentissage, on décide de supprime tout les exemples de la base de données dont la valeur du champ Size est 'Varies with device'.

In [ ]:
gpsdf.drop(gpsdf[gpsdf['Size'] == "Varies with device"].index, inplace=True)

Le nombre d'exemples dans la base de données est désormais de :

In [ ]:
print(gpsdf.shape[0], "exemples")

On aimerait que ces valeurs soit des flottants et non pas des String. On suppose que les 'M' représente les mega-octets et 'k' les kilo-octets.
<br/>On effectue alors une modification générale sur tout les champs Size des exemples de la base de données. <br/>On décide de garder comme valeur la taille en mega-octets de l'application.

In [ ]:
size_data = gpsdf['Size'].loc[gpsdf['Size'].str.contains('k')].index.tolist()
convert_data = pd.DataFrame(gpsdf.loc[size_data, 'Size'].apply(lambda x : x.strip('k')).astype(float).apply(lambda x : x/1024).apply(lambda x : round(x, 3)).astype(str))
gpsdf.loc[size_data, 'Size'] = convert_data

gpsdf['Size'] = gpsdf['Size'].apply(lambda x:x.strip('M'))
gpsdf['Size'] = gpsdf['Size'].astype(float)

On affiche les valeurs uniques de la colonne Size après modification.

In [ ]:
gpsdf.loc[:,'Size'].unique()

### Nettoyage de la colonne Installs

Dans la colonne Installs, les données sont représentées sous la forme "valeur"+ et la valeur est parfois séparée par des virgules. On préfère garder uniquement la valeur et retirer le symbole +. On fait donc appel à la fonction remove_elements sur la colonne.

Les éléments uniques de la colonne Installs avant l'utilisation de la fonction sont :

In [ ]:
gpsdf.loc[:,'Installs'].unique()

In [ ]:
gpsdf['Installs'] = clean_elements(gpsdf['Installs'])

Les éléments uniques de la colonne Installs après l'utilisation de la fonction sont :

In [ ]:
gpsdf.loc[:,'Installs'].unique()

On souhaite également transformer les valeurs de Installs en entier afin de faciliter leurs utilisations lors de l'apprentissage (elles sont pour l'instant au format String).
<br/>On utilise pour cela la fonction to_numeric de la bibliothèque pandas.

In [ ]:
gpsdf['Installs'] = pd.to_numeric(gpsdf['Installs'])
gpsdf.loc[:,'Installs'].unique()

### Nettoyage de la colonne Type

On affiche les valeurs uniques de la colonne Type.

In [ ]:
gpsdf.loc[:,'Type'].unique()

Afin que ces données soient utilisables lors de l'apprentissage, on décide de représenter la valeur 'Free' par la valeur 0 et la valeur 'Paid' par la valeur 1.

In [ ]:
gpsdf.loc[:,'Type'] = gpsdf.loc[:,'Type'].apply(lambda x : 0 if (x == 'Free') else 1)

On affiche les valeurs uniques de la colonne Type après modification.

In [ ]:
gpsdf.loc[:,'Type'].unique()

### Nettoyage de la colonne Price

On affiche les valeurs uniques de la colonne Price.

In [ ]:
gpsdf.loc[:,'Price'].unique()

Afin que ces valeurs soient utilisables lors de l'apprentissage, on décide de retirer les symboles '$' et de les convertir en flottant.

In [ ]:
gpsdf['Price'] = clean_elements(gpsdf['Price'])
gpsdf['Price'] = pd.to_numeric(gpsdf['Price'])

On affiche les valeurs uniques de la colonne Price après modification.

In [ ]:
gpsdf.loc[:,'Price'].unique()

### Nettoyage de la colonne Content Rating

On affiche les valeurs uniques de la colonne Content Rating.

In [ ]:
gpsdf.loc[:,'Content Rating'].unique()

Afin que ces valeurs soient utilisables lors de l'apprentissage, on décide de diviser la colonne 'Content Rating' en plusieurs colonnes différents. 
<br/>Chaque colonne permettra de savoir à quelle catégorie de 'Content Rating' cette application appartient.
<br/>(Par exemple, si une application a la valeur 1 dans son champ 'CR_Everyone', cela signifie que l'application est destinée à tout les public, soit 'Everyone').

In [ ]:
gpsdf = pd.concat([gpsdf, pd.get_dummies(gpsdf['Content Rating'], prefix='CR')], axis=1)
gpsdf.drop(['Content Rating'], axis=1, inplace=True)

### Nettoyage des colonnes Application, Genres, Last Updated, Current Ver et Android Ver

Aucune modification ne sera effectuée au niveau de ces colonnes puisque leurs données ne seront pas utilisées lors de l'apprentissage.

On affiche l'état de la base de données après avoir réalisé la modification.



In [ ]:
gpsdf.head()

Les différentes colonnes de la base de données sont désormais :

In [ ]:
for c in gpsdf.columns :
    print(c)
print("(" + str(gpsdf.shape[1]) + " champs différents)")

## Données Google Play Store Reviews

Comme pour la première base de données que nous allons utiliser, nous allons procéder au nettoyage de la deuxième afin que ces données soient utilisables lors de l'apprentissage.

On récupère la base de données :

In [ ]:
gpsrevdf = pd.read_csv("data/GoogleApps/googleplaystore_user_reviews.csv")
gpsrevdf[0:5]

Le nombre d'exemple dans la base de données est de :

In [ ]:
print(str(gpsrevdf.shape[0]) + " exemples")

Les différentes champs des exemples de la base de données sont :

In [ ]:
for c in gpsrevdf.columns :
    print(c)
print("(" + str(gpsrevdf.shape[1]) + " champs différents)")

On affiche les informations de la base de données :

In [ ]:
gpsrevdf.info()

On remarque que les champs Translated_Review, Sentiment, Sentiment_Polarity ou Sentiment_Subjectivity de certains exemples sont nuls. Il faut donc retirer ces exemples en priorité.

On s'occupe dans un premier temps de retirer toutes les exemples possedant des champs avec des valeurs Nan.

In [ ]:
gpsrevdf = gpsrevdf.dropna()
gpsrevdf.info()

Le nombre d'exemples dans la base de données est désormais de 37427 exemples.

On compte dans cette base de données :

In [ ]:
print("- " + str(len(gpsrevdf[gpsrevdf['Sentiment'] == 'Positive'])) + " revues positives")
print("- " + str(len(gpsrevdf[gpsrevdf['Sentiment'] == 'Neutral'])) + " revues sans sentiments particuliers")
print("- " + str(len(gpsrevdf[gpsrevdf['Sentiment'] == 'Negative'])) + " revues négatives")

## Partie 1 - Description du problème

Aujourd'hui, le téléphone portable est un appareil dont on ne peut plus se passer. Elles peuvent remplir un nombre illimité de fonctions. Afin de remplir ces fonctions, des applications sont développées et sont à télécharger sur des marchés d'applications (GooglePlayStore pour les produits Android, ...).

On va voir quels sont les facteurs qui permette de prédire la popularité d'une application mobile ? Est-ce que le fait qu'une application soit payante ou non puisse empêcher la prédiction de sa popularité ?

## Partie 2 - Modèle

Le modèle utilisé afin de répondre à la problématique est le classifieur k-means.

La particularité de ce modèle est de regrouper les exemples d'une base de données au sein de clusters. Les clusters sont des ensembles d'exemples situés autour d'un centroïde.

À l'initialisation, les centroïdes sont des exemples tirés de la base de données d'apprentissage et on définit les premiers clusters. Puis à chaque itération durant n itérations, on met à jour les clusters et un dictionnaire d'appartenance. Ce dictionnaire d'appartenance permet de savoir à quel cluster chaque exemple appartient. Il est important de noter qu'un exemple ne peut appartenir qu'à un seul et unique cluster.

Après avoir réalisé l'apprentissage sur un certain nombre d'itérations, on obtient alors un ensemble de clusters fixes. On peut alors prédire des exemples grace au modèle. Comme on a pas eu à définir une méthode de prédiction précise lors du TME8, on a décidé de notre propre méthode de prédiction.

La prédiction d'un exemple est réalisée de la manière suivante :
- on place l'exemple à prédire dans nos données
- on trouve à quel cluster il appartient
- on récupère les points faisant partie de ce cluster grace au dictionnaire d'appartenance
- on renvoie le label le plus représenté dans le cluster

## Partie 3 - Code

### Mise en place de l'algorithme d'apprentissage

Le code du classifieur utilisant cet algorithme pour son apprentissage est le suivant :

In [ ]:
# Toutes les fonctions liées à ce classifieur ont été mises en place dans le fichier utils.py du package iads
class ClassifierKmeans(cl.Classifier):
    """ Classe pour représenter un classifieur linéaire aléatoire
        Cette classe hérite de la classe Classifier
    """
    def __init__(self, nbCentroides):
        """ Constructeur de Classifier
            Argument:
                - intput_dimension (int) : dimension de la description des exemples
            Hypothèse : input_dimension > 0
        """
        self.nbCentroides = nbCentroides
        self.desc_set = []
        self.label_set = []
        self.liste_centroides = []
        self.matrice_affectation = []
        
    def train(self, desc_set, label_set, epsilon=0.01, itermax=1000):
        """ Permet d'entrainer le modele sur l'ensemble donné
            desc_set: ndarray avec des descriptions
            label_set: ndarray avec les labels correspondants
            Hypothèse: desc_set et label_set ont le même nombre de lignes
        """
        self.desc_set = desc_set
        self.label_set = label_set
        # Utilisation de l'algorithme des k-means
        (centroides, affectation) = ut.kmoyennes(self.nbCentroides, self.desc_set, epsilon, itermax)
        self.liste_centroides = centroides
        self.matrice_affectation = affectation
            
    def score(self,x):
        """ renvoie un dictionnaire de prédiction sur x (valeur réelle)
            x: une description
        """
        id_centroide = ut.plus_proche(x, self.liste_centroides)
        liste_attr = self.matrice_affectation[id_centroide]
        
        # On effectue le compte sur les labels présents dans la liste
        compte_label = dict()
        for p in liste_attr :
            if (self.label_set[p] not in compte_label.keys()) :
                compte_label[self.label_set[p]] = 1 
            else :
                compte_label[self.label_set[p]] += 1 

        return compte_label
    
    def predict(self, x):
        """ rend la prediction sur x (soit -1 ou soit +1)
            x: une description
        """
        dict_prediction = self.score(x)
        keys_dict = list(dict_prediction.keys())
        
        # On cherche le label le plus présent dans le cluster associé à la description x
        prediction = keys_dict[0]
        for k in keys_dict :
            if (dict_prediction[prediction] < dict_prediction[k]) :
                prediction = k
                
        return prediction

### Visualisation des données

Grâce à la libraire Seaborn (importée sous le nom 'sns'), on peut avoir une idée de l'état des bases de données que nous allons utiliser.

On affiche dans un premier temps les données importantes de la première base de données, soit gpsdf, en faisait la séparation entre les données concernant les applications payantes et les applications gratuites.

On considère que les données principales de cette base de données sont :
- Rating
- Size
- Installs
- Reviews
- Type
- Price

In [ ]:
# On récupère les données importantes de la base de données gpsdf
rating_data = gpsdf['Rating']
size_data = gpsdf['Size']
installs_data = gpsdf['Installs']
reviews_data = gpsdf['Reviews']
type_data = gpsdf['Type']
price_data = gpsdf['Price']

# Affichage des données
plotgrap = sns.pairplot(pd.DataFrame(list(zip(rating_data, size_data, np.log(installs_data), np.log(reviews_data), type_data, price_data)), columns=['Rating', 'Size', 'Installs', 'Reviews', 'Type', 'Price']), hue='Type', palette="Set1")

Rappel de la signification des valeurs de Type :
- 0 = Free
- 1 = Paid

On affiche désormais les données importantes de la deuxième base de données, soit gpsrevdf, en faisait la séparation entre les données concernant les revues positives et negatives.

On considère que les données principales de cette base de données sont :
- Sentiment
- Sentiment_Polarity
- Sentiment_Subjectivity

In [ ]:
# On récupère les données importantes de la base de données gpsrevdf 
sentiment_data = gpsrevdf['Sentiment']
sent_pol_data = gpsrevdf['Sentiment_Polarity']
sent_sub_data = gpsrevdf['Sentiment_Subjectivity']

# Affichage des données
plotgrap = sns.pairplot(pd.DataFrame(list(zip(sentiment_data, sent_pol_data, sent_sub_data)), columns=['Sentiment', 'Sentiment_Polarity', 'Sentiment_Subjectivity']), hue='Sentiment', palette="Set1")

## Partie 4 - Protocole expérimental

Le protocole expérimental est le suivant :

#### Etape 1
On met en place des modèles de prédiction basés le nombre de téléchargements (Installs) de chaque application de la base de données.
<br/>Pour chaque apprentissage, la combinaison des paramètres utilisés, soit les dimensions utilisées pour classer les exemples, seront différentes.
<br/>Afin de vérifier les résultats de chaque classifieur, on utilise une validation croisée.

#### Etape 2
On fait le même travail qu'à l'étape précédente, sauf que les modèles de prédiction seront cette fois-ci basés sur la note (Rating) de chaque application de la base de données.

#### Etape 3
On effectue une première analyse des résultats obtenus dans les étapes 1 et 2, soit les précisions de la validation croisée.
<br/>On trouve quel modèle de prédiction est le plus prometteur en matière de résultats. Tous les modèles générés par la suite seront basés sur ce modèle.
<br/>On effectue également une sélection des exemples dans la base de données. On retire les exemples que l'on considère comme "inutiles", soit les exemples n'ayant aucune valeur d'apprentissage à nos yeux, et on vérifie le nouvel état de la base de données.

#### Etape 4
On met en place de nouveaux modèles de prédiction en utilisant la nouvelle base de données créée à l'étape 3. Les résultats rendus par ces modèles doivent normalement être supérieurs à ceux obtenus dans les étapes précédentes puisque nous avons réalisé une sélection des exemples en ne conservant que les plus intéressants.

#### Etape 5
On récupère le modèle ayant réalisé les meilleures performances. On renouvelle alors l'apprentissage sur ce modèle sur plusieurs itérations avec un nombre de centroïdes différents à chaque itération. On compare par la suite les résultats obtenus et on conserve le nouveau modèle avec les meilleures performances.

#### Etape 6
On effectue une séparation de la base de données afin de pouvoir utiliser le modèle sur les applications payantes et les applications gratuites. On effectue une nouvelle série de tests avec les deux nouveaux ensembles de données.

#### Remarque sur l'expérimentation
Chaque fois que l'on doit mettre en place un nouveau modèle de prédiction, on crée dans un premier temps un modèle pilote, avec un important nombre de centroïde, dont on affichera la représentation graphique afin d'avoir une idée de la disposition des données et leur répartition au sein des clusters.

## Partie 5 - Résultats

## 1) Prédiction du nombre de téléchargements (Installs)

Dans cette première partie des résultats, on va chercher à montrer s'il existe une combinaison de facteurs permettant une prédiction du nombre de téléchargements d'une application.
<br/>Les facteurs allant être utilisés ici sont :
- la note attribuée à l'application (Rating)
- le nombre d'avis déposés par les utilisateurs (Reviews)
- la taille en octets d'une application (Size)

### Prédiction en fonction de Rating et Reviews

In [ ]:
# Récupération des labels
label_installs_set = gpsdf['Installs'].to_numpy()

# Récupération des descriptions
desc_rating_reviews_set = ut.normalisation(gpsdf[['Rating', 'Reviews']].to_numpy()) # Rating x Reviews

# Création d'un classifieur k-means allant étudier la situation
kmeans_rating_reviews = ClassifierKmeans(30)

# Entrainement du classifieur
kmeans_rating_reviews.train(desc_rating_reviews_set, label_installs_set, epsilon=0.001)

Après avoir réalisé l'apprentissage, on obtient les clusters suivants :

In [ ]:
ut.affiche_resultat(kmeans_rating_reviews.desc_set ,kmeans_rating_reviews.liste_centroides, kmeans_rating_reviews.matrice_affectation)

(chaque couleur représente un cluster et chaque croix représente un centroide de cluster)

En utilisant 50 clusters différents, on obtient les résultats de précision suivants grâce à la validation croisée sur 10 itérations :

In [ ]:
# Récupération des données (Rating X Reviews => Installs)
X = ut.normalisation(gpsdf[['Rating', 'Reviews']].to_numpy())
Y = gpsdf['Installs'].to_numpy()

niter = 10
nbCentroides = 50
perf = []
# Utilisation de la validation croisée
for i in range(niter):
    Xapp,Yapp,Xtest,Ytest = ut.crossval(X, Y, niter, i)
    cl = ClassifierKmeans(nbCentroides)
    cl.train(Xapp, Yapp)
    perf.append(cl.accuracy(Xtest, Ytest))
    print("Apprentissage ",i+1,":\t"," |Yapp|= ",len(Yapp)," |Ytest|= ",len(Ytest),"\tperf= ",perf[-1])

# On transforme la liste en array numpy pour avoir les fonctions statistiques:
perf = np.array(perf)
print(f'\nRésultat global avec crossval :\tmoyenne= {perf.mean():.3f}\técart-type= {perf.std():.3f}')

### Prédiction en fonction de Rating et Size

In [ ]:
# Récupération des labels
label_installs_set = gpsdf['Installs'].to_numpy()

# Récupération des descriptions
desc_rating_size_set = ut.normalisation(gpsdf[['Rating', 'Size']].to_numpy()) # Rating x Size

# Création d'un classifieur k-means allant étudier la situation
kmeans_rating_size = ClassifierKmeans(50)

# Entrainement du classifieur
kmeans_rating_size.train(desc_rating_size_set, label_installs_set, epsilon=0.001)

Après avoir réalisé l'apprentissage, on obtient les clusters suivants :

In [ ]:
ut.affiche_resultat(kmeans_rating_size.desc_set ,kmeans_rating_size.liste_centroides, kmeans_rating_size.matrice_affectation)

En utilisant 50 clusters différents, on obtient les résultats de précision suivants grâce à la validation croisée sur 10 itérations :

In [ ]:
# Récupération des données (Rating X Size => Installs)
X = ut.normalisation(gpsdf[['Rating', 'Size']].to_numpy())
Y = gpsdf['Installs'].to_numpy()

niter = 10
nbCentroides = 50
perf = []
# Utilisation de la validation croisée
for i in range(niter):
    Xapp,Yapp,Xtest,Ytest = ut.crossval(X, Y, niter, i)
    cl = ClassifierKmeans(nbCentroides)
    cl.train(Xapp, Yapp)
    perf.append(cl.accuracy(Xtest, Ytest))
    print("Apprentissage ",i+1,":\t"," |Yapp|= ",len(Yapp)," |Ytest|= ",len(Ytest),"\tperf= ",perf[-1])

# On transforme la liste en array numpy pour avoir les fonctions statistiques:
perf = np.array(perf)
print(f'\nRésultat global avec crossval :\tmoyenne= {perf.mean():.3f}\técart-type= {perf.std():.3f}')

### Prédiction en fonction de Reviews et Size

In [ ]:
# Récupération des labels
label_installs_set = gpsdf['Installs'].to_numpy()

# Récupération des descriptions
desc_reviews_size_set = ut.normalisation(gpsdf[['Reviews', 'Size']].to_numpy()) # Rating x Size

# Création d'un classifieur k-means allant étudier la situation
kmeans_reviews_size = ClassifierKmeans(50)

# Entrainement du classifieur
kmeans_reviews_size.train(desc_reviews_size_set, label_installs_set, epsilon=0.001)

Après avoir réalisé un premier apprentissage, on obtient les clusteurs suivants :

In [ ]:
ut.affiche_resultat(kmeans_reviews_size.desc_set ,kmeans_reviews_size.liste_centroides, kmeans_reviews_size.matrice_affectation)

En utilisant 50 clusters différents, on obtient les résultats de précision suivants grâce à la validation croisée sur 10 itérations :

In [ ]:
# Récupération des données (Reviews X Size => Installs)
X = ut.normalisation(gpsdf[['Reviews', 'Size']].to_numpy())
Y = gpsdf['Installs'].to_numpy()

niter = 10
nbCentroides = 50
perf = []
# Utilisation de la validation croisée
for i in range(niter):
    Xapp,Yapp,Xtest,Ytest = ut.crossval(X, Y, niter, i)
    cl = ClassifierKmeans(nbCentroides)
    cl.train(Xapp, Yapp)
    perf.append(cl.accuracy(Xtest, Ytest))
    print("Apprentissage ",i+1,":\t"," |Yapp|= ",len(Yapp)," |Ytest|= ",len(Ytest),"\tperf= ",perf[-1])

# On transforme la liste en array numpy pour avoir les fonctions statistiques:
perf = np.array(perf)
print(f'\nRésultat global avec crossval :\tmoyenne= {perf.mean():.3f}\técart-type= {perf.std():.3f}')

## 2) Prédiction de la note de l'application (Rating)

On s'intéresse désormais à la note moyenne donnée par les utilisateurs de l'application à cette dernière. On va chercher s'il existe une combinaison de facteurs permettant de pouvoir prédire la note d'une application.
<br/>Les facteurs allant être étudiés dans cette partie sont :
- le nombre de téléchargements de l'application (Installs)
- le nombre d'avis déposés par les utilisateurs (Reviews)
- la taille en octets d'une application (Size)

### Prédiction en fonction de Installs et Reviews

In [ ]:
# Récupération des labels
label_rating_set = gpsdf['Rating'].to_numpy()

# Récupération des descriptions
desc_installs_reviews_set = ut.normalisation(gpsdf[['Installs', 'Reviews']].to_numpy()) # Installs x Reviews

# Création d'un classifieur k-means allant étudier la situation
kmeans_installs_reviews = ClassifierKmeans(50)

# Entrainement du classifieur
kmeans_installs_reviews.train(desc_installs_reviews_set, label_rating_set, epsilon=0.001)

Après avoir réalisé un premier apprentissage, on obtient les clusters suivants :

In [ ]:
ut.affiche_resultat(kmeans_installs_reviews.desc_set, kmeans_installs_reviews.liste_centroides, kmeans_installs_reviews.matrice_affectation)

En utilisant 50 clusters différents, on obtient les résultats de précision suivants grâce à la validation croisée sur 10 itérations :

In [ ]:
# Récupération des données (Installs X Reviews => Rating)
X = ut.normalisation(gpsdf[['Installs', 'Reviews']].to_numpy())
Y = gpsdf['Rating'].to_numpy()

niter = 10
nbCentroides = 50
perf = []
# Utilisation de la validation croisée
for i in range(niter):
    Xapp,Yapp,Xtest,Ytest = ut.crossval(X, Y, niter, i)
    cl = ClassifierKmeans(nbCentroides)
    cl.train(Xapp, Yapp)
    perf.append(cl.accuracy(Xtest, Ytest))
    print("Apprentissage ",i+1,":\t"," |Yapp|= ",len(Yapp)," |Ytest|= ",len(Ytest),"\tperf= ",perf[-1])

# On transforme la liste en array numpy pour avoir les fonctions statistiques:
perf = np.array(perf)
print(f'\nRésultat global avec crossval :\tmoyenne= {perf.mean():.3f}\técart-type= {perf.std():.3f}')

### Prédiction en fonction de Installs et Size

In [ ]:
# Récupération des labels
label_rating_set = gpsdf['Rating'].to_numpy()

# Récupération des descriptions
desc_installs_size_set = ut.normalisation(gpsdf[['Installs', 'Size']].to_numpy()) # Installs x Reviews

# Création d'un classifieur k-means allant étudier la situation
kmeans_installs_size = ClassifierKmeans(50)

# Entrainement du classifieur
kmeans_installs_size.train(desc_installs_size_set, label_rating_set, epsilon=0.001)

Après avoir réalisé un premier apprentissage, on obtient les clusters suivants :

In [ ]:
ut.affiche_resultat(kmeans_installs_size.desc_set, kmeans_installs_size.liste_centroides, kmeans_installs_size.matrice_affectation)

En utilisant 50 clusters différents, on obtient les résultats de précision suivants grâce à la validation croisée sur 10 itérations :

In [ ]:
# Récupération des données (Installs X Size => Rating)
X = ut.normalisation(gpsdf[['Installs', 'Size']].to_numpy())
Y = gpsdf['Rating'].to_numpy()

niter = 10
nbCentroides = 50
perf = []
# Utilisation de la validation croisée
for i in range(niter):
    Xapp,Yapp,Xtest,Ytest = ut.crossval(X, Y, niter, i)
    cl = ClassifierKmeans(nbCentroides)
    cl.train(Xapp, Yapp)
    perf.append(cl.accuracy(Xtest, Ytest))
    print("Apprentissage ",i+1,":\t"," |Yapp|= ",len(Yapp)," |Ytest|= ",len(Ytest),"\tperf= ",perf[-1])

# On transforme la liste en array numpy pour avoir les fonctions statistiques:
perf = np.array(perf)
print(f'\nRésultat global avec crossval :\tmoyenne= {perf.mean():.3f}\técart-type= {perf.std():.3f}')

### Prédiction en fonction de Size et Reviews

In [ ]:
# Récupération des labels
label_rating_set = gpsdf['Rating'].to_numpy()

# Récupération des descriptions
desc_size_reviews_set = ut.normalisation(gpsdf[['Size', 'Reviews']].to_numpy()) # Installs x Reviews

# Création d'un classifieur k-means allant étudier la situation
kmeans_size_reviews = ClassifierKmeans(50)

# Entrainement du classifieur
kmeans_size_reviews.train(desc_size_reviews_set, label_rating_set, epsilon=0.001)

Après avoir réalisé un premier apprentissage, on obtient les clusters suivants :

In [ ]:
ut.affiche_resultat(kmeans_size_reviews.desc_set, kmeans_size_reviews.liste_centroides, kmeans_size_reviews.matrice_affectation)

En utilisant 50 clusters différents, on obtient les résultats de précision suivants grâce à la validation croisée sur 10 itérations :

In [ ]:
# Récupération des données (Size x Reviews => Rating)
X = ut.normalisation(gpsdf[['Size', 'Reviews']].to_numpy())
Y = gpsdf['Rating'].to_numpy()

niter = 10
nbCentroides = 50
perf = []
# Utilisation de la validation croisée
for i in range(niter):
    Xapp,Yapp,Xtest,Ytest = ut.crossval(X, Y, niter, i)
    cl = ClassifierKmeans(nbCentroides)
    cl.train(Xapp, Yapp)
    perf.append(cl.accuracy(Xtest, Ytest))
    print("Apprentissage ",i+1,":\t"," |Yapp|= ",len(Yapp)," |Ytest|= ",len(Ytest),"\tperf= ",perf[-1])

# On transforme la liste en array numpy pour avoir les fonctions statistiques:
perf = np.array(perf)
print(f'\nRésultat global avec crossval :\tmoyenne= {perf.mean():.3f}\técart-type= {perf.std():.3f}')

## 3) Analyse des premiers résultats

Après avoir réalisé une première série d'analyses, on en arrive à une première conclusion : on ne peut pas prédire la popularité d'une application en fonction de sa note (Rating) puisque ces premiers résultats montrent une précision grâce à la validation croisée d'environ 12% en moyenne, ce qui n'est pas suffisant.

On va donc se concentrer uniquement sur le nombre de téléchargements de l'application.

Il y a un trop grand nombre d'applications avec un faible nombre de téléchargements mais à la note élevée. Il faudrait donc réaliser une sélection dans la base de données et conserver uniquement les applications les plus populaires. On considère qu'une application n'est populaire que si elle remplit les critères suivants :
- un nombre de téléchargements supérieur ou égal à 1 million
- une note moyenne donnée par les utilisateurs de l'application supérieure ou égale à 4

On réalise alors cette sélection sur la base de données.

In [ ]:
# On récupère l'indice des applications satisfaisant les conditions citées au-dessus
newgpsdf = gpsdf[gpsdf['Rating'] >= 4.] # Note supérieure ou égale à 4
newgpsdf = newgpsdf[newgpsdf['Installs'] >= 10000000] # Nombre de téléchargements supérieur ou égal à 1.000.000

Le nombre d'exemples restant dans cette base de données est de :

In [ ]:
print(newgpsdf.shape[0], "exemples.")

## 4) Prédiction du nombre de téléchargements (Installs) dans la nouvelle base de données

Ayant désormais une base de données plus adaptée à nos recherches, on effectue une nouvelle fois une série de tests afin de mettre en avant s'il existe un moyen de prédire le nombre de téléchargements d'une application à travers une série de facteurs.

### Prédiction en fonction de Rating et Reviews

In [ ]:
# Récupération des labels
label_installs_set = newgpsdf['Installs'].to_numpy()

# Récupération des descriptions
desc_rating_reviews_set = ut.normalisation(newgpsdf[['Rating', 'Reviews']].to_numpy()) # Rating x Reviews

# Création d'un classifieur k-means allant étudier la situation
kmeans_rating_reviews = ClassifierKmeans(30)

# Entrainement du classifieur
kmeans_rating_reviews.train(desc_rating_reviews_set, label_installs_set, epsilon=0.001)

Après avoir réalisé un premier apprentissage, on obtient les clusters suivants :

In [ ]:
ut.affiche_resultat(kmeans_rating_reviews.desc_set, kmeans_rating_reviews.liste_centroides, kmeans_rating_reviews.matrice_affectation)

En utilisant **(len(np.unique(Y)) * 2)** centroides différents, on obtient les résultats de précision suivants grâce à la validation croisée sur 10 itérations :

In [ ]:
# Récupération des données (Rating x Reviews => Installs)
X = ut.normalisation(newgpsdf[['Rating', 'Reviews']].to_numpy())
Y = newgpsdf['Installs'].to_numpy()

niter = 10
nbCentroides = len(np.unique(Y)) * 2
perf = []

# Utilisation de la validation croisée
for i in range(niter) :
    Xapp,Yapp,Xtest,Ytest = ut.crossval(X, Y, niter, i)
    cl = ClassifierKmeans(nbCentroides)
    cl.train(Xapp, Yapp)
    perf.append(cl.accuracy(Xtest, Ytest))
    print("Apprentissage ",i+1,":\t"," |Yapp|= ",len(Yapp)," |Ytest|= ",len(Ytest),"\tperf= ",perf[-1])

# On transforme la liste en array numpy pour avoir les fonctions statistiques:
perf = np.array(perf)
print(f'\nRésultat global avec crossval :\tmoyenne= {perf.mean():.3f}\técart-type= {perf.std():.3f}')

### Prédiction en fonction de Rating et Size

In [ ]:
# Récupération des labels
label_installs_set = newgpsdf['Installs'].to_numpy()

# Récupération des descriptions
desc_rating_size_set = ut.normalisation(newgpsdf[['Rating', 'Size']].to_numpy()) # Rating x Reviews

# Création d'un classifieur k-means allant étudier la situation
kmeans_rating_size = ClassifierKmeans(30)

# Entrainement du classifieur
kmeans_rating_size.train(desc_rating_size_set, label_installs_set, epsilon=0.001)

Après avoir réalisé un premier apprentissage, on obtient les clusters suivants :

In [ ]:
ut.affiche_resultat(kmeans_rating_size.desc_set, kmeans_rating_size.liste_centroides, kmeans_rating_size.matrice_affectation)

En utilisant **(len(np.unique(Y)) * 2)** centroides différents, on obtient les résultats de précision suivants grâce à la validation croisée sur 10 itérations :

In [ ]:
# Récupération des données (Rating x Size => Installs)
X = ut.normalisation(newgpsdf[['Rating', 'Size']].to_numpy())
Y = newgpsdf['Installs'].to_numpy()

niter = 10
nbCentroides = len(np.unique(Y)) * 2
perf = []

# Utilisation de la validation croisée
for i in range(niter) :
    Xapp,Yapp,Xtest,Ytest = ut.crossval(X, Y, niter, i)
    cl = ClassifierKmeans(nbCentroides)
    cl.train(Xapp, Yapp)
    perf.append(cl.accuracy(Xtest, Ytest))
    print("Apprentissage ",i+1,":\t"," |Yapp|= ",len(Yapp)," |Ytest|= ",len(Ytest),"\tperf= ",perf[-1])

# On transforme la liste en array numpy pour avoir les fonctions statistiques:
perf = np.array(perf)
print(f'\nRésultat global avec crossval :\tmoyenne= {perf.mean():.3f}\técart-type= {perf.std():.3f}')

### Prédiction en fonction de Reviews et Size

In [ ]:
# Récupération des labels
label_installs_set = newgpsdf['Installs'].to_numpy()

# Récupération des descriptions
desc_reviews_size_set = ut.normalisation(newgpsdf[['Reviews', 'Size']].to_numpy()) # Rating x Reviews

# Création d'un classifieur k-means allant étudier la situation
kmeans_reviews_size = ClassifierKmeans(30)

# Entrainement du classifieur
kmeans_reviews_size.train(desc_reviews_size_set, label_installs_set, epsilon=0.001)

Après avoir réalisé un premier apprentissage, on obtient les clusters suivants :

In [ ]:
ut.affiche_resultat(kmeans_reviews_size.desc_set, kmeans_reviews_size.liste_centroides, kmeans_reviews_size.matrice_affectation)

En utilisant **(len(np.unique(Y)) * 2)** centroides différents, on obtient les résultats de précision suivants grâce à la validation croisée sur 10 itérations :

In [ ]:
# Récupération des données (Reviews x Size => Installs)
X = ut.normalisation(newgpsdf[['Reviews', 'Size']].to_numpy())
Y = newgpsdf['Installs'].to_numpy()

niter = 10
nbCentroides = len(np.unique(Y)) * 2
perf = []

# Utilisation de la validation croisée
for i in range(niter) :
    Xapp,Yapp,Xtest,Ytest = ut.crossval(X, Y, niter, i)
    cl = ClassifierKmeans(nbCentroides)
    cl.train(Xapp, Yapp)
    perf.append(cl.accuracy(Xtest, Ytest))
    print("Apprentissage ",i+1,":\t"," |Yapp|= ",len(Yapp)," |Ytest|= ",len(Ytest),"\tperf= ",perf[-1])

# On transforme la liste en array numpy pour avoir les fonctions statistiques:
perf = np.array(perf)
print(f'\nRésultat global avec crossval :\tmoyenne= {perf.mean():.3f}\técart-type= {perf.std():.3f}')

### Prédiction en fonction de Rating, Size et Reviews

On essaye maintenant d'utiliser les 3 facteurs précédents en meme temps.

En utilisant **(len(np.unique(Y)) * 2)** centroides différents, on obtient les résultats de précision suivants grâce à la validation croisée sur 10 itérations :

In [ ]:
# Récupération des données (Reviews x Rating x Size => Installs)
X = ut.normalisation(newgpsdf[['Reviews', 'Rating', 'Size']].to_numpy())
Y = newgpsdf['Installs'].to_numpy()

niter = 10
nbCentroides = len(np.unique(Y)) * 2
perf = []

# Utilisation de la validation croisée
for i in range(niter) :
    Xapp,Yapp,Xtest,Ytest = ut.crossval(X, Y, niter, i)
    cl = ClassifierKmeans(nbCentroides)
    cl.train(Xapp, Yapp)
    perf.append(cl.accuracy(Xtest, Ytest))
    print("Apprentissage ",i+1,":\t"," |Yapp|= ",len(Yapp)," |Ytest|= ",len(Ytest),"\tperf= ",perf[-1])

# On transforme la liste en array numpy pour avoir les fonctions statistiques:
perf = np.array(perf)
print(f'\nRésultat global avec crossval :\tmoyenne= {perf.mean():.3f}\técart-type= {perf.std():.3f}')

### 5) Utilisation du meilleur modèle trouvé avec un nombre de centroides différent

Le modèle ayant présenté les meilleurs résultats lors de nos résultats est le modèle réalisant la prédiction du nombre d'installations d'une application en fonction de son nombre d'avis et sa note dans la base de données modifiée (précision moyenne de 74.4%).

Nous allons donc réaliser de nouveaux apprentissages sur ce modèle en utilisant à chaque fois un nombre de centroides différents. A chaque itération, on affichera la moyenne de précision l'écart-type en fonction du nombre de centroides.

In [ ]:
# Récupération des données (Rating x Reviews => Installs)
X = ut.normalisation(newgpsdf[['Rating', 'Reviews']].to_numpy())
Y = newgpsdf['Installs'].to_numpy()

# Parametres utilisés (on incrementera le nombre de centroides de 5 à chaque fois sur l'intervalle 5 et 50)
# On utilise toujours 10 de crossval pour les résultats
nbCentroides = 5
niter = 10

# Utilisation de la validation croisée
while (nbCentroides <= 50) :
    
    # Réalisation des tests
    perf = []
    for i in range(niter) :
        Xapp,Yapp,Xtest,Ytest = ut.crossval(X, Y, niter, i)
        cl = ClassifierKmeans(nbCentroides)
        cl.train(Xapp, Yapp)
        perf.append(cl.accuracy(Xtest, Ytest))
    
    # Affichage et incrémentation
    perf = np.array(perf)
    print(f'Résultat global avec {nbCentroides} centroides :\tmoyenne= {perf.mean():.3f}\técart-type= {perf.std():.3f}')
    nbCentroides += 5

### 6) Nouvel apprentissage en fonction du caractère payant ou non de l'application

On récupère les applications gratuites et les applications payantes de la base de données newgpsdf, soit les applications payantes parmi les applications ayant plus d'un million de téléchargements et une note moyenne supérieure à 4.

In [ ]:
# On récupère les applications gratuites de newgpsdf
newgpsdf_free = newgpsdf[newgpsdf['Type'] == 0]
print("Le nombre d'applications gratuites est de " + str(newgpsdf_free.shape[0]) + ".")

# On récupère les applications payantes de newgpsdf
newgpsdf_paid = newgpsdf[newgpsdf['Type'] == 1]
print("Le nombre d'applications payantes est de " + str(newgpsdf_paid.shape[0]) + ".")

On remarque alors directement qu'il n'existe qu'un seul exemple dans la base de données représentant une application payante au nombre de téléchargements supérieurs à un million et à la note moyenne supérieure à 4.<br/>Cela rend donc impossible donc la réalisation de tout apprentissage et prédiction sur les applications payantes.

Le nombre d'exemples représentant des applications gratuites étant le même que le nombre d'exemples dans la base de données d'origine, à 1 près, on en conclut que l'apprentissage et la prédiction peut être réalisée sans problème.

## Partie 6 - Analyse

Les premiers résultats ont permis de mettre en avant qu'il était envisageable qu'un modèle utilisant une combinaison de facteurs précis puisse permettre de prédire la popularité d'une application. La popularité était représentée soit par le nombre de téléchargements, soit par la note moyenne de l'application.

Les modèles qu'on a décidé d'utiliser consistent à prédire le nombre d'installations d'une application en fonction de la combinaison de facteurs choisie, avec une précision moyenne de 22%. Prédire la note moyenne de l'application paraissait impossible puisque la précision moyenne des modèles testés réalisant cette prédiction était proche de 12%.

Les premiers résultats n'étant pas satisfaisants, on a donc réalisé une sélection au sein des exemples de la base de données afin de ne conserver que les applications populaires.

On a considéré qu'une application est considérée comme populaire que si son nombre de téléchargements était supérieur à 1 million et sa note moyenne supérieure à 4. On a choisi ces critères-là puisqu' on suppose qu'une personne sur PlayStore sera intéressée par une application remplissant ces deux critères.

Par la suite, on a utilisé les modèles de prédiction du nombre d'installations avec la nouvelle base de données. Les résultats obtenus étaient nettement supérieurs à ceux obtenus précédemment. La précision moyenne des modèles est passé de 22% à 70%. Il s'agit donc importante amélioration.

Le meilleur modèle est celui réalisant la prédiction du nombre d'installations en fonction du nombre d'avis et de la note moyenne de l'application, avec une précision moyenne de 75%. Il est donc possible de prédire le nombre d'installations d'une application populaire 3 fois sur 4.

On a ensuite cherché à savoir si le fait qu'une application soit payante ou non puisse influencer la prédiction du nombre d'installations. Il se trouve que parmi les applications qu'on a jugées populaires, seulement une seule est payante. On en conclut que notre modèle ne permet pas de prédire le nombre d'installations des applications populaires payantes.